<a href="https://colab.research.google.com/github/abhinav70291/TechnicalAnalysis-for-intra-day-trading/blob/main/Technical_Analysis_rectified.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade ipywidgets matplotlib
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import interact, SelectMultiple, SelectionRangeSlider
from datetime import datetime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.4/139.4 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 91.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 95.2 MB/s eta 0:00:00


In [2]:
df=pd.read_csv("/content/Manappuram_10minute.csv")
df.tail(5)

,Date,Open,High,Low,Close,Volume
2492,2020-04-28T14:35:00,123.90,124.25,123.7,124.15,289726
2493,2020-04-28T14:45:00,124.15,125.80,124.1,125.80,1706683
2494,2020-04-28T14:55:00,125.80,125.80,125.8,125.80,145240
2495,2020-04-28T15:05:00,125.80,131.50,125.8,130.85,3162404
2496,2020-04-28T15:15:00,130.95,130.95,129.6,129.75,221556


In [3]:
df["pricing"]=df.Close * df.Volume *0.01

In [4]:
df.pricing.max(),df.pricing.min()

(22709551.2825, 9449.505000000001)

In [5]:
# Converting Date column into datetime dftype
df['Date'] = pd.to_datetime(df['Date'])
df.set_index('Date', inplace=True)


# **State the Values you want to consider for this backtesting and technical analysis**

In [6]:
# Initialize a DataFrame to store metrics for each parameter
metrics_df = pd.DataFrame(columns=['indicator', 'hit_rate', 'profit_factor', 'sharpe_ratio', 'max_drawdown', 'total_profit', 'profit_percentage', 'avg_win_to_avg_loss_ratio', 'sortino_ratio', 'num_trades'])

# **Defining ALL indicator functions  here**

In [7]:
# calculates SMA using "n" as the rolling window size
def ma_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  return df

# calculates SMA using "m" and "n" as the rolling window size
def longshort_ma_calc(df, m, n):
  df["sma_short"] = df.Close.rolling(window=min(m,n)).mean()
  df["sma_long"] = df.Close.rolling(window=max(m,n)).mean()
  return df

# calculates EMA using "n" as the rolling window size
def ema_calc(df, n):
  df["ema"] = df.Close.ewm(span=n, adjust=False).mean()
  return df


def longshort_ema_calc(df, m, n):
  df["ema_short"] = df.Close.ewm(span=min(m,n), adjust=False).mean()
  df["ema_long"] = df.Close.ewm(span=max(m,n), adjust=False).mean()
  return df

# calculates RSI using "n" as the lookback period
def rsi_calc(df, n):
  df['rsi'] = 100 - (100 / (1 + df['Close'].diff().apply(lambda x: x if x > 0 else 0).rolling(window=n).mean() / df['Close'].diff().apply(lambda x: -x if x < 0 else 0).rolling(window=n).mean()))
  return df

# calculates OBV
def obv_calc(df):
  df['obv'] = (np.sign(df['Close'].diff()) * df['Volume']).fillna(0).cumsum()
  return df

def bb_calc(df, n):
  df["sma"] = df.Close.rolling(window=n).mean()
  df["std"] = df.Close.rolling(window=n).std()
  df["upper_bb"] = df["sma"] + (2 * df["std"])
  df["lower_bb"] = df["sma"] - (2 * df["std"])
  return df

# Volume weighted average price
def vwap_calc(df):
    df['vwap'] = (df['Volume'] * df['Close']).cumsum() / df['Volume'].cumsum()
    return df

# Supertrend Indicator
def supertrend_calc(df, period, multiplier):
    # Calculate basic upper and lower bands
    df['hl_avg'] = (df['High'] + df['Low']) / 2
    df['range'] = df['High'] - df['Low']
    df['upper_band'] = df['hl_avg'] + multiplier * df['range']
    df['lower_band'] = df['hl_avg'] - multiplier * df['range']

    # Calculate final upper and lower bands
    df['upper_band_final'] = np.where((df['upper_band'] < df['upper_band'].shift(1)) | (df['Close'] > df['upper_band'].shift(1)), df['upper_band'], df['upper_band'].shift(1))
    df['lower_band_final'] = np.where((df['lower_band'] > df['lower_band'].shift(1)) | (df['Close'] < df['lower_band'].shift(1)), df['lower_band'], df['lower_band'].shift(1))

    # Calculate Supertrend
    df['supertrend'] = np.where(df['Close'] <= df['upper_band_final'], df['upper_band_final'], df['lower_band_final'])
    df['supertrend'] = np.where(df['Close'] >= df['lower_band_final'], df['lower_band_final'], df['supertrend'])

    return df

# calculates Average Directional Index (ADX)
def adx_calc(df, n):
    df['hl_diff'] = df['High'] - df['Low']
    df['hc_diff'] = abs(df['High'] - df['Close'].shift(1))
    df['lc_diff'] = abs(df['Low'] - df['Close'].shift(1))
    df['tr'] = df[['hl_diff', 'hc_diff', 'lc_diff']].max(axis=1)
    df['+dm'] = np.where((df['High'] > df['High'].shift(1)) & (df['High'] - df['High'].shift(1) > df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
    df['-dm'] = np.where((df['Low'] < df['Low'].shift(1)) & (df['High'].shift(1) - df['High'] < df['Low'].shift(1) - df['Low']), df['Low'].shift(1) - df['Low'], 0)
    df['tr_ema'] = df['tr'].ewm(span=n, adjust=False).mean()
    df['+dm_ema'] = df['+dm'].ewm(span=n, adjust=False).mean()
    df['-dm_ema'] = df['-dm'].ewm(span=n, adjust=False).mean()
    df['+di'] = (df['+dm_ema'] / df['tr_ema']) * 100
    df['-di'] = (df['-dm_ema'] / df['tr_ema']) * 100
    df['dx'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100
    df['adx'] = df['dx'].rolling(window=n).mean()

    return df

# calculates MACD
def macd_calc(df, short_n, long_n, signal_n):
    df['ema_short'] = df['Close'].ewm(span=short_n, adjust=False).mean()
    df['ema_long'] = df['Close'].ewm(span=long_n, adjust=False).mean()
    df['macd_line'] = df['ema_short'] - df['ema_long']
    df['signal_line'] = df['macd_line'].ewm(span=signal_n, adjust=False).mean()
    df['macd_histogram'] = df['macd_line'] - df['signal_line']

    return df


# **Backtesting starts from here**
**Our Indicators for consideration here are -**

(i)  SMA

(ii) Short term SMA and Long term SMA

(iii) EMA

(iv) Short term EMA and Long term EMA

(v) RSI

(vi) Average Directional Index

(vii)  Bollinger Bands



In [9]:
# Run the loop and find the optimal parameters
starting_capital = 10000  # Starting capital in your currency
stop_loss = 0.05  # Stop loss level (5% in this case)
upper_cap = 8000  # Upper investment cap in your currency
lower_cap = 1000  # Lower investment cap in your currency
transaction_cost = 0.001  # Transaction cost (0.1% in this case)
leverage = 1 # Leverage (1x in this case meaning no leverage, just the original capital)
volume_percentage = 0.001  # Minimum average daily volume

**SMA**

In [45]:
metrics_df=pd.DataFrame()

In [46]:
import numpy as np
import pandas as pd

def ma_calc(df, n):
    df['sma'] = df['Close'].rolling(n).mean()
    return df

def backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage):
    df = ma_calc(df, n)
    df['sma_signal'] = 0  # Initialize the signal column
    in_position = False
    capital = starting_capital
    num_trades = 0
    wins = []
    losses = []
    daily_profits=[]

    for day in np.unique(df.index.date):
        df_day = df[df.index.date == day]
        profits = []
        for index, row in df_day.iterrows():
            volume_limit = row.Volume * volume_percentage
            if not in_position and row.Close > row.sma:
                buyprice = row.Close
                shares_to_trade = min(volume_limit, upper_cap // buyprice)
                investment = buyprice * shares_to_trade

                if investment > capital * leverage:
                    continue

                capital -= investment * (1 + transaction_cost)
                in_position = True
                df.loc[index, 'sma_signal'] = 1  # Indicate a buy decision

            if in_position and (row.Close < row.sma and row.Close < buyprice * (1 - stop_loss)) :
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'sma_signal'] = -1  # Indicate a sell decision
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)

            # If still in position at the end of the day, sell
            if in_position and index == df_day.index[-1]:
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'sma_signal'] = -1  # Indicate a sell decision
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)

        daily_profits.append(sum(profits))



    daily_profits = pd.Series(daily_profits)
    hit_rate = (daily_profits > 0).mean()
    profit_factor = daily_profits[daily_profits > 0].sum() / -daily_profits[daily_profits < 0].sum()
    sharpe_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits.std()
    sortino_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits[daily_profits < 0].std()
    max_drawdown = np.min(daily_profits)
    total_profit = capital - starting_capital
    profit_percentage = (total_profit / starting_capital) * 100
    avg_win_to_avg_loss_ratio = np.abs(np.mean(wins) / np.mean(losses))

    return df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades

metrics_dict_sma = {}

for n in range(304, 570):
    _, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
    print(f"For parameter: {n}, Hit Rate: {hit_rate}, Profit Factor: {profit_factor}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}, Total Profit: {total_profit}, Profit Percentage: {profit_percentage}%, Average Win to Average Loss ratio: {avg_win_to_avg_loss_ratio}, Sortino Ratio: {sortino_ratio}, Number of Trades: {num_trades}")
    metrics_dict_sma[n] = (hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades)

# Find the best n
best_n = max(metrics_dict_sma, key=lambda x: metrics_dict_sma[x][2])  # This line finds the n that gives the maximum Sharpe Ratio
print("The best window size is ", best_n)
# Run the backtest function with the best n to get the sma_signal column
df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades =backtest(df, best_n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
# Assuming metrics_df is your metrics report DataFrame
metrics_df = metrics_df.append({
    "Indicator":"SMA",
    'Best_n': best_n,
    'Best_m': None,
    'Hit_Rate': hit_rate,
    'Profit_Factor': profit_factor,
    'Sharpe_Ratio': sharpe_ratio,
    'Max_Drawdown': max_drawdown,
    'Total_Profit': total_profit,
    'Profit_Percentage': profit_percentage,
    'Avg_Win_to_Avg_Loss_Ratio': avg_win_to_avg_loss_ratio,
    'Sortino_Ratio': sortino_ratio,
    'Num_Trades': num_trades
}, ignore_index=True)



For parameter: 304, Hit Rate: 0.22727272727272727, Profit Factor: 1.4263239931851956, Sharpe Ratio: 4.504034067246991, Max Drawdown: -490.0000000000009, Total Profit: 682.9377000000204, Profit Percentage: 6.829377000000204%, Average Win to Average Loss ratio: 1.333560831926434, Sortino Ratio: 5.5875792465479845, Number of Trades: 31
For parameter: 305, Hit Rate: 0.22727272727272727, Profit Factor: 1.4263239931851956, Sharpe Ratio: 4.504034067246991, Max Drawdown: -490.0000000000009, Total Profit: 698.5669000000198, Profit Percentage: 6.985669000000199%, Average Win to Average Loss ratio: 1.2446567764646719, Sortino Ratio: 5.5875792465479845, Number of Trades: 30
For parameter: 306, Hit Rate: 0.22727272727272727, Profit Factor: 1.4423753665689147, Sharpe Ratio: 4.628410030424003, Max Drawdown: -490.0000000000009, Total Profit: 728.9218000000183, Profit Percentage: 7.289218000000183%, Average Win to Average Loss ratio: 1.258535522301322, Sortino Ratio: 5.6723485273399, Number of Trades: 

In [47]:
metrics_df.head()

,Indicator,Best_n,Best_m,Hit_Rate,Profit_Factor,Sharpe_Ratio,Max_Drawdown,Total_Profit,Profit_Percentage,Avg_Win_to_Avg_Loss_Ratio,Sortino_Ratio,Num_Trades
0,SMA,527,None,0.181818,1.980214,7.296833,-411.4,1438.2956,14.382956,1.485161,10.209546,21


**Long term and Short term SMA**

In [49]:
import numpy as np
import pandas as pd

def longshort_ma_calc(df, m, n):
    df['sma_short'] = df['Close'].rolling(m).mean()
    df['sma_long'] = df['Close'].rolling(n).mean()
    return df

volume_threshold = 1.5 * df['Volume'].mean()

def backtest(df, m, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage):
    df['sma_crossover_signal'] = 0
    df = longshort_ma_calc(df, m, n)
    in_position = False
    capital = starting_capital
    num_trades = 0
    wins = []
    losses = []
    daily_profits=[]

    for day in np.unique(df.index.date):
        df_day = df[df.index.date == day]
        profits = []
        for index, row in df_day.iterrows():
            volume_limit = row.Volume * volume_percentage
            if not in_position and row.sma_short > row.sma_long:
                buyprice = row.Close
                shares_to_trade = min(volume_limit, upper_cap // buyprice)
                investment = buyprice * shares_to_trade

                if investment > capital * leverage:
                    continue

                capital -= investment * (1 + transaction_cost)
                in_position = True
                df.loc[index, 'sma_crossover_signal'] = 1

            if in_position and (row.sma_short < row.sma_long and row.Close < buyprice * (1 - stop_loss)):
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'sma_crossover_signal'] = -1
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)

            # If still in position at the end of the day, sell
            if in_position and index == df_day.index[-1]:
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'sma_crossover_signal'] = -1
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)


        daily_profits.append(sum(profits))

    daily_profits = pd.Series(daily_profits)
    hit_rate = (daily_profits > 0).mean()
    profit_factor = daily_profits[daily_profits > 0].sum() / -daily_profits[daily_profits < 0].sum()
    sharpe_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits.std()
    sortino_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits[daily_profits < 0].std()
    max_drawdown = np.min(daily_profits)
    total_profit = capital - starting_capital
    profit_percentage = (total_profit / starting_capital) * 100
    avg_win_to_avg_loss_ratio = np.mean(wins) / np.mean(losses)

    return df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades

metrics_dict_ls_sma = {}

for m in range(2, 19):
    for n in range(38*8, 38*20+1,10):
        df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, m, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
        print(f"For parameter: ({m},{n}), Hit Rate: {hit_rate}, Profit Factor: {profit_factor}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}, Total Profit: {total_profit}, Profit Percentage: {profit_percentage}%, Average Win to Average Loss ratio: {avg_win_to_avg_loss_ratio}, Sortino Ratio: {sortino_ratio}, Number of Trades: {num_trades}")
        metrics_dict_ls_sma[(m, n)] = (hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades)

# Find the best (m, n)
best_m_n = max(metrics_dict_ls_sma, key=lambda x: metrics_dict_ls_sma[x][2])  # This line finds the (m, n) that gives the maximum Sharpe Ratio
print("The best parameters are ", best_m_n)

# Run the backtest function with the best (m, n) to get the sma_signal column
df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, best_m_n[0], best_m_n[1], starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)

metrics_df = metrics_df.append({
    "Indicator":"SMA_Crossover",
    'Best_n': best_m_n[1],  # best_n is now the second element of best_m_n
    'Best_m': best_m_n[0],  # best_m is now the first element of best_m_n
    'Hit_Rate': hit_rate,
    'Profit_Factor': profit_factor,
    'Sharpe_Ratio': sharpe_ratio,
    'Max_Drawdown': max_drawdown,
    'Total_Profit': total_profit,
    'Profit_Percentage': profit_percentage,
    'Avg_Win_to_Avg_Loss_Ratio': avg_win_to_avg_loss_ratio,
    'Sortino_Ratio': sortino_ratio,
    'Num_Trades': num_trades
}, ignore_index=True)


For parameter: (2,304), Hit Rate: 0.22727272727272727, Profit Factor: 1.4072285603700911, Sharpe Ratio: 4.4375965867715275, Max Drawdown: -402.5499999999993, Total Profit: 627.8246500000241, Profit Percentage: 6.278246500000241%, Average Win to Average Loss ratio: -1.4072285603700911, Sortino Ratio: 6.451994330336291, Number of Trades: 30
For parameter: (2,314), Hit Rate: 0.22727272727272727, Profit Factor: 1.5062252927919606, Sharpe Ratio: 5.302491816102868, Max Drawdown: -402.5499999999993, Total Profit: 908.758200000022, Profit Percentage: 9.08758200000022%, Average Win to Average Loss ratio: -1.4058102732724964, Sortino Ratio: 7.984831525499154, Number of Trades: 29
For parameter: (2,324), Hit Rate: 0.24242424242424243, Profit Factor: 1.8150470219435735, Sharpe Ratio: 7.4354785448894765, Max Drawdown: -384.8000000000002, Total Profit: 1383.7676000000247, Profit Percentage: 13.837676000000245%, Average Win to Average Loss ratio: -1.4747257053291534, Sortino Ratio: 11.450457387228983

**EMA**

In [66]:
import numpy as np
import pandas as pd

def ma_calc(df, n):
    df['sma'] = df['Close'].rolling(n).mean()
    return df

def backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage):
    df = ema_calc(df, n)
    df['ema_signal'] = 0  # Initialize the signal column
    in_position = False
    capital = starting_capital
    num_trades = 0
    wins = []
    losses = []
    daily_profits=[]

    for day in np.unique(df.index.date):
        df_day = df[df.index.date == day]
        profits = []
        for index, row in df_day.iterrows():
            volume_limit = row.Volume * volume_percentage
            if not in_position and row.Close > row.ema:
                buyprice = row.Close
                shares_to_trade = min(volume_limit, upper_cap // buyprice)
                investment = buyprice * shares_to_trade

                if investment > capital * leverage:
                    continue

                capital -= investment * (1 + transaction_cost)
                in_position = True
                df.loc[index, 'ema_signal'] = 1  # Indicate a buy decision

            if in_position and (row.Close < row.ema and row.Close < buyprice * (1 - stop_loss)) :
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'ema_signal'] = -1  # Indicate a sell decision
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)

            # If still in position at the end of the day, sell
            if in_position and index == df_day.index[-1]:
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'ema_signal'] = -1  # Indicate a sell decision
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)

        daily_profits.append(sum(profits))



    daily_profits = pd.Series(daily_profits)
    hit_rate = (daily_profits > 0).mean()
    profit_factor = daily_profits[daily_profits > 0].sum() / -daily_profits[daily_profits < 0].sum()
    sharpe_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits.std()
    sortino_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits[daily_profits < 0].std()
    max_drawdown = np.min(daily_profits)
    total_profit = capital - starting_capital
    profit_percentage = (total_profit / starting_capital) * 100
    avg_win_to_avg_loss_ratio = np.abs(np.mean(wins) / np.mean(losses))

    return df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades

metrics_dict_ema = {}

for n in range(38, 570):
    _, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
    print(f"For parameter: {n}, Hit Rate: {hit_rate}, Profit Factor: {profit_factor}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}, Total Profit: {total_profit}, Profit Percentage: {profit_percentage}%, Average Win to Average Loss ratio: {avg_win_to_avg_loss_ratio}, Sortino Ratio: {sortino_ratio}, Number of Trades: {num_trades}")
    metrics_dict_ema[n] = (hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades)

# Find the best n
best_n = max(metrics_dict_ema, key=lambda x: metrics_dict_ema[x][2])  # This line finds the n that gives the maximum Sharpe Ratio
print("The best window size is ", best_n)
# Run the backtest function with the best n to get the sma_signal column
df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades =backtest(df, best_n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
# Assuming metrics_df is your metrics report DataFrame
metrics_df = metrics_df.append({
    "Indicator":"EMA",
    'Best_n': best_n,
    'Best_m': None,
    'Hit_Rate': hit_rate,
    'Profit_Factor': profit_factor,
    'Sharpe_Ratio': sharpe_ratio,
    'Max_Drawdown': max_drawdown,
    'Total_Profit': total_profit,
    'Profit_Percentage': profit_percentage,
    'Avg_Win_to_Avg_Loss_Ratio': avg_win_to_avg_loss_ratio,
    'Sortino_Ratio': sortino_ratio,
    'Num_Trades': num_trades
}, ignore_index=True)



For parameter: 38, Hit Rate: 0.36363636363636365, Profit Factor: 1.621555793357823, Sharpe Ratio: 6.385886570580734, Max Drawdown: -959.800000000002, Total Profit: 1896.9230534500566, Profit Percentage: 18.969230534500568%, Average Win to Average Loss ratio: 1.724086887717055, Sortino Ratio: 9.85924021183517, Number of Trades: 54
For parameter: 39, Hit Rate: 0.3484848484848485, Profit Factor: 1.5029635031421718, Sharpe Ratio: 5.408162528737156, Max Drawdown: -959.800000000002, Total Profit: 1507.4034034500582, Profit Percentage: 15.074034034500583%, Average Win to Average Loss ratio: 1.6000722273062664, Sortino Ratio: 8.19213677906306, Number of Trades: 54
For parameter: 40, Hit Rate: 0.3484848484848485, Profit Factor: 1.6956257450614314, Sharpe Ratio: 7.049625653052193, Max Drawdown: -494.9499999999998, Total Profit: 2089.769253450053, Profit Percentage: 20.89769253450053%, Average Win to Average Loss ratio: 1.7423499057498815, Sortino Ratio: 13.768123784988996, Number of Trades: 53
F

**Long short EMA**

In [71]:
import numpy as np
import pandas as pd


def backtest(df, m, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage):
    df['ema_crossover_signal'] = 0
    df = longshort_ema_calc(df, m, n)
    in_position = False
    capital = starting_capital
    num_trades = 0
    wins = []
    losses = []
    daily_profits=[]

    for day in np.unique(df.index.date):
        df_day = df[df.index.date == day]
        profits = []
        for index, row in df_day.iterrows():
            volume_limit = row.Volume * volume_percentage
            if not in_position and row.ema_short > row.ema_long:
                buyprice = row.Close
                shares_to_trade = min(volume_limit, upper_cap // buyprice)
                investment = buyprice * shares_to_trade

                if investment > capital * leverage:
                    continue

                capital -= investment * (1 + transaction_cost)
                in_position = True
                df.loc[index, 'ema_crossover_signal'] = 1

            if in_position and (row.ema_short < row.ema_long and row.Close < buyprice * (1 - stop_loss)):
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'ema_crossover_signal'] = -1
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)

            # If still in position at the end of the day, sell
            if in_position and index == df_day.index[-1]:
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'ema_crossover_signal'] = -1
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)


        daily_profits.append(sum(profits))

    daily_profits = pd.Series(daily_profits)
    hit_rate = (daily_profits > 0).mean()
    profit_factor = daily_profits[daily_profits > 0].sum() / -daily_profits[daily_profits < 0].sum()
    sharpe_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits.std()
    sortino_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits[daily_profits < 0].std()
    max_drawdown = np.min(daily_profits)
    total_profit = capital - starting_capital
    profit_percentage = (total_profit / starting_capital) * 100
    avg_win_to_avg_loss_ratio = np.mean(wins) / np.mean(losses)

    return df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades

metrics_dict_ls_ema = {}

for m in range(5, 38):
    for n in range(38*8, 38*20+1,10):
        df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, m, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
        print(f"For parameter: ({m},{n}), Hit Rate: {hit_rate}, Profit Factor: {profit_factor}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}, Total Profit: {total_profit}, Profit Percentage: {profit_percentage}%, Average Win to Average Loss ratio: {avg_win_to_avg_loss_ratio}, Sortino Ratio: {sortino_ratio}, Number of Trades: {num_trades}")
        metrics_dict_ls_ema[(m, n)] = (hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades)

# Find the best (m, n)
best_m_n = max(metrics_dict_ls_ema, key=lambda x: metrics_dict_ls_ema[x][4])  # This line finds the (m, n) that gives the maximum Sharpe Ratio
print("The best parameters are ", best_m_n)

# Run the backtest function with the best (m, n) to get the sma_signal column
df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, best_m_n[0], best_m_n[1], starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)

metrics_df = metrics_df.append({
    "Indicator":"EMA_Crossover",
    'Best_n': best_m_n[1],  # best_n is now the second element of best_m_n
    'Best_m': best_m_n[0],  # best_m is now the first element of best_m_n
    'Hit_Rate': hit_rate,
    'Profit_Factor': profit_factor,
    'Sharpe_Ratio': sharpe_ratio,
    'Max_Drawdown': max_drawdown,
    'Total_Profit': total_profit,
    'Profit_Percentage': profit_percentage,
    'Avg_Win_to_Avg_Loss_Ratio': avg_win_to_avg_loss_ratio,
    'Sortino_Ratio': sortino_ratio,
    'Num_Trades': num_trades
}, ignore_index=True)


For parameter: (5,304), Hit Rate: 0.2727272727272727, Profit Factor: 1.3796895563287617, Sharpe Ratio: 4.317722010692498, Max Drawdown: -492.0, Total Profit: 513.0861500000319, Profit Percentage: 5.13086150000032%, Average Win to Average Loss ratio: -1.226390716736677, Sortino Ratio: 5.717362007880004, Number of Trades: 34
For parameter: (5,314), Hit Rate: 0.25757575757575757, Profit Factor: 1.3960729262278866, Sharpe Ratio: 4.488193562596664, Max Drawdown: -492.0, Total Profit: 560.220650000032, Profit Percentage: 5.60220650000032%, Average Win to Average Loss ratio: -1.3960729262278866, Sortino Ratio: 5.905106718412229, Number of Trades: 34
For parameter: (5,324), Hit Rate: 0.25757575757575757, Profit Factor: 1.3864203963562547, Sharpe Ratio: 4.3763920220947545, Max Drawdown: -492.0, Total Profit: 534.9925500000354, Profit Percentage: 5.349925500000354%, Average Win to Average Loss ratio: -1.3864203963562547, Sortino Ratio: 5.784318689759057, Number of Trades: 34
For parameter: (5,33

**RSI**

In [52]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

def backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage):
  df = rsi_calc(df, n)
  df['rsi_signal'] = 0
  in_position = False
  daily_profits = []
  capital = starting_capital
  net_profit_loss = 0
  num_trades = 0
  wins = []
  losses = []

  for day in np.unique(df.index.date):
    df_day = df[df.index.date == day]
    profits = []
    for index, row in df_day.iterrows():
        volume_limit = row.Volume * volume_percentage  # Set volume limit as a percentage of daily volume
        if not in_position:
            if row.rsi < 30 and row.Volume > volume_limit:  # RSI is less than 30, oversold condition
                buyprice = row.Close
                shares_to_trade = min(volume_limit, upper_cap // buyprice)
                investment = buyprice * shares_to_trade

                if investment > capital * leverage:
                    continue

                capital -= investment * (1 + transaction_cost)
                in_position = True
                df.loc[index, 'rsi_signal'] = 1

        if in_position:
            if row.rsi > 70 or row.Close < buyprice * (1 - stop_loss):  # RSI is greater than 70, overbought condition
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'rsi_signal'] = -1
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)
    # If still in position at the end of the day, sell
        if in_position and index == df_day.index[-1]:
            sell_proceeds = row.Close * shares_to_trade
            profit = sell_proceeds - investment
            profits.append(profit)
            capital += sell_proceeds * (1 - transaction_cost)
            in_position = False
            num_trades += 1
            df.loc[index, 'rsi_signal'] = -1
            if profit > 0:
                wins.append(profit)
            else:
                losses.append(profit)

    daily_profits.append(sum(profits))

  # Calculate metrics
  daily_profits = pd.Series(daily_profits)
  hit_rate = (daily_profits > 0).mean()
  profit_factor = daily_profits[daily_profits > 0].sum() / -daily_profits[daily_profits < 0].sum()
  sharpe_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits.std()
  sortino_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits[daily_profits < 0].std()
  max_drawdown = np.min(daily_profits)
  total_profit = capital - starting_capital
  profit_percentage = (total_profit / starting_capital) * 100
  avg_win_to_avg_loss_ratio = np.mean(wins) / np.mean(losses)

  return df,hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades

# Initialize a dictionary to store metrics for each parameter
metrics_dict_rsi = {}

for n in range(38,38*10):
    _, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
    print(f"For parameter: {n}, Hit Rate: {hit_rate}, Profit Factor: {profit_factor}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}, Total Profit: {total_profit}, Profit Percentage: {profit_percentage}%, Average Win to Average Loss ratio: {avg_win_to_avg_loss_ratio}, Sortino Ratio: {sortino_ratio}, Number of Trades: {num_trades}")
    metrics_dict_rsi[n] = (hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades)

# Find the best n
best_n = max(metrics_dict_rsi, key=lambda x: metrics_dict_rsi[x][2])  # This line finds the n that gives the maximum Sharpe Ratio
print("The best window size is ", best_n)
# Run the backtest function with the best n to get the sma_signal column
df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades =backtest(df, best_n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
# Assuming metrics_df is your metrics report DataFrame
metrics_df = metrics_df.append({
    "Indicator":"RSI",
    'Best_n': best_n,
    'Best_m': None,
    'Hit_Rate': hit_rate,
    'Profit_Factor': profit_factor,
    'Sharpe_Ratio': sharpe_ratio,
    'Max_Drawdown': max_drawdown,
    'Total_Profit': total_profit,
    'Profit_Percentage': profit_percentage,
    'Avg_Win_to_Avg_Loss_Ratio': avg_win_to_avg_loss_ratio,
    'Sortino_Ratio': sortino_ratio,
    'Num_Trades': num_trades
}, ignore_index=True)



For parameter: 38, Hit Rate: 0.07575757575757576, Profit Factor: 0.9982990593798364, Sharpe Ratio: -0.013496109415302115, Max Drawdown: -1457.9999999999982, Total Profit: -290.7967999999855, Profit Percentage: -2.9079679999998556%, Average Win to Average Loss ratio: -1.5688594923966932, Sortino Ratio: -0.00849587281315967, Number of Trades: 18
For parameter: 39, Hit Rate: 0.07575757575757576, Profit Factor: 0.8919515984208203, Sharpe Ratio: -0.9693925664737575, Max Drawdown: -1457.9999999999982, Total Profit: -711.7519499999871, Profit Percentage: -7.1175194999998705%, Average Win to Average Loss ratio: -1.7909814031072946, Sortino Ratio: -0.6275571990541814, Number of Trades: 21
For parameter: 40, Hit Rate: 0.07575757575757576, Profit Factor: 0.9841049105807764, Sharpe Ratio: -0.13516256662046447, Max Drawdown: -1457.9999999999982, Total Profit: -370.5320499999834, Profit Percentage: -3.705320499999834%, Average Win to Average Loss ratio: -1.8286362023582614, Sortino Ratio: -0.0861519

**Bollinger Bands**

In [55]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

def backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage):
  df = bb_calc(df, n)
  df['bb_signal'] = 0
  in_position = False
  daily_profits = []
  capital = starting_capital
  net_profit_loss = 0
  num_trades = 0
  wins = []
  losses = []

  for day in np.unique(df.index.date):
    df_day = df[df.index.date == day]
    profits = []
    for index, row in df_day.iterrows():
      volume_limit = row.Volume * volume_percentage  # Set volume limit as a percentage of daily volume
      if not in_position:
        if row.Close > row.upper_bb and row.Volume > volume_limit:  # Price is above lower Bollinger Band
            buyprice = row.Close
            shares_to_trade = min(volume_limit, upper_cap // buyprice)
            investment = buyprice * shares_to_trade

            if investment > capital * leverage:
                continue

            capital -= investment * (1 + transaction_cost)
            in_position = True
            df.loc[index, 'bb_signal'] = 1

      if in_position:
        if row.Close < row.lower_bb and row.Close < buyprice * (1 - stop_loss):  # Price is above upper Bollinger Band or stop loss is hit
            sell_proceeds = row.Close * shares_to_trade
            profit = sell_proceeds - investment
            profits.append(profit)
            capital += sell_proceeds * (1 - transaction_cost)
            in_position = False
            num_trades += 1
            df.loc[index, 'bb_signal'] = -1
            if profit > 0:
                wins.append(profit)
            else:
                losses.append(profit)
        if in_position and index == df_day.index[-1]:
            sell_proceeds = row.Close * shares_to_trade
            profit = sell_proceeds - investment
            profits.append(profit)
            capital += sell_proceeds * (1 - transaction_cost)
            in_position = False
            num_trades += 1
            df.loc[index, 'bb_signal'] = -1
            if profit > 0:
                wins.append(profit)
            else:
                losses.append(profit)

    daily_profits.append(sum(profits))

  # Calculate metrics
  daily_profits = pd.Series(daily_profits)
  hit_rate = (daily_profits > 0).mean()
  profit_factor = daily_profits[daily_profits > 0].sum() / -daily_profits[daily_profits < 0].sum()
  sharpe_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits.std()
  sortino_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits[daily_profits < 0].std()
  max_drawdown = np.min(daily_profits)
  total_profit = capital - starting_capital
  profit_percentage = (total_profit / starting_capital) * 100
  avg_win_to_avg_loss_ratio = np.mean(wins) / np.mean(losses)

  return df,hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades

# Initialize a dictionary to store metrics for each parameter
metrics_dict_bb = {}
for n in range(2, 38*10):
    _, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
    print(f"For parameter: {n}, Hit Rate: {hit_rate}, Profit Factor: {profit_factor}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}, Total Profit: {total_profit}, Profit Percentage: {profit_percentage}%, Average Win to Average Loss ratio: {avg_win_to_avg_loss_ratio}, Sortino Ratio: {sortino_ratio}, Number of Trades: {num_trades}")
    metrics_dict_bb[n] = (hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades)

# Find the best n
best_n = max(metrics_dict_bb, key=lambda x: metrics_dict_bb[x][4])  # This line finds the n that gives the maximum Sharpe Ratio
print("The best window size is ", best_n)
# Run the backtest function with the best n to get the sma_signal column
df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades =backtest(df, best_n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
# Assuming metrics_df is your metrics report DataFrame
metrics_df = metrics_df.append({
    "Indicator":"Bollinger Bands",
    'Best_n': best_n,
    'Best_m': None,
    'Hit_Rate': hit_rate,
    'Profit_Factor': profit_factor,
    'Sharpe_Ratio': sharpe_ratio,
    'Max_Drawdown': max_drawdown,
    'Total_Profit': total_profit,
    'Profit_Percentage': profit_percentage,
    'Avg_Win_to_Avg_Loss_Ratio': avg_win_to_avg_loss_ratio,
    'Sortino_Ratio': sortino_ratio,
    'Num_Trades': num_trades
}, ignore_index=True)


For parameter: 2, Hit Rate: 0.0, Profit Factor: nan, Sharpe Ratio: nan, Max Drawdown: 0, Total Profit: 0, Profit Percentage: 0.0%, Average Win to Average Loss ratio: nan, Sortino Ratio: nan, Number of Trades: 0
For parameter: 3, Hit Rate: 0.0, Profit Factor: nan, Sharpe Ratio: nan, Max Drawdown: 0, Total Profit: 0, Profit Percentage: 0.0%, Average Win to Average Loss ratio: nan, Sortino Ratio: nan, Number of Trades: 0
For parameter: 4, Hit Rate: 0.0, Profit Factor: nan, Sharpe Ratio: nan, Max Drawdown: 0, Total Profit: 0, Profit Percentage: 0.0%, Average Win to Average Loss ratio: nan, Sortino Ratio: nan, Number of Trades: 0
For parameter: 5, Hit Rate: 0.0, Profit Factor: nan, Sharpe Ratio: nan, Max Drawdown: 0, Total Profit: 0, Profit Percentage: 0.0%, Average Win to Average Loss ratio: nan, Sortino Ratio: nan, Number of Trades: 0
For parameter: 6, Hit Rate: 0.045454545454545456, Profit Factor: 2.0883612995358884, Sharpe Ratio: 4.05012990510641, Max Drawdown: -440.1999999999998, Total

**ADX**

In [54]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

def adx_calc(df, n):
    df['hl_diff'] = df['High'] - df['Low']
    df['hc_diff'] = abs(df['High'] - df['Close'].shift(1))
    df['lc_diff'] = abs(df['Low'] - df['Close'].shift(1))
    df['tr'] = df[['hl_diff', 'hc_diff', 'lc_diff']].max(axis=1)
    df['+dm'] = np.where((df['High'] > df['High'].shift(1)) & (df['High'] - df['High'].shift(1) > df['Low'].shift(1) - df['Low']), df['High'] - df['High'].shift(1), 0)
    df['-dm'] = np.where((df['Low'] < df['Low'].shift(1)) & (df['High'].shift(1) - df['High'] < df['Low'].shift(1) - df['Low']), df['Low'].shift(1) - df['Low'], 0)
    df['tr_ema'] = df['tr'].ewm(span=n, adjust=False).mean()
    df['+dm_ema'] = df['+dm'].ewm(span=n, adjust=False).mean()
    df['-dm_ema'] = df['-dm'].ewm(span=n, adjust=False).mean()
    df['+di'] = (df['+dm_ema'] / df['tr_ema']) * 100
    df['-di'] = (df['-dm_ema'] / df['tr_ema']) * 100
    df['dx'] = (abs(df['+di'] - df['-di']) / (df['+di'] + df['-di'])) * 100
    df['adx'] = df['dx'].rolling(window=n).mean()

    return df

def backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage):
  df = adx_calc(df, n)
  df['adx_signal'] = 0
  in_position = False
  daily_profits = []
  capital = starting_capital
  net_profit_loss = 0
  num_trades = 0
  wins = []
  losses = []

  for day in np.unique(df.index.date):
    df_day = df[df.index.date == day]
    profits = []
    for index, row in df_day.iterrows():
        volume_limit = row.Volume * volume_percentage  # Set volume limit as a percentage of daily volume
        if not in_position:
            if row['+di'] > row['-di'] and row.adx > 25 and row.Volume > volume_limit:
                buyprice = row.Close
                shares_to_trade = min(volume_limit, upper_cap // buyprice)
                investment = buyprice * shares_to_trade

                if investment > capital * leverage:
                    continue

                capital -= investment * (1 + transaction_cost)
                in_position = True
                df.loc[index, 'adx_signal'] = 1

        if in_position:
            if row['+di'] < row['-di'] or row.Close < buyprice * (1 - stop_loss):
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'adx_signal'] = -1
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)
        if in_position and index == df_day.index[-1]:
                sell_proceeds = row.Close * shares_to_trade
                profit = sell_proceeds - investment
                profits.append(profit)
                capital += sell_proceeds * (1 - transaction_cost)
                in_position = False
                num_trades += 1
                df.loc[index, 'adx_signal'] = -1
                if profit > 0:
                    wins.append(profit)
                else:
                    losses.append(profit)

    daily_profits.append(sum(profits))

  # Calculate metrics
  daily_profits = pd.Series(daily_profits)
  hit_rate = (daily_profits > 0).mean()
  profit_factor = daily_profits[daily_profits > 0].sum() / -daily_profits[daily_profits < 0].sum()
  sharpe_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits.std()
  sortino_ratio = np.sqrt(len(df)) * daily_profits.mean() / daily_profits[daily_profits < 0].std()
  max_drawdown = np.min(daily_profits)
  total_profit = capital - starting_capital
  profit_percentage = (total_profit / starting_capital) * 100
  avg_win_to_avg_loss_ratio = np.mean(wins) / np.mean(losses)

  return df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades


# Initialize a dictionary to store metrics for each parameter
metrics_dict_adx = {}

for n in range(2, 38*10):
    _, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades = backtest(df, n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
    print(f"For parameter: {n}, Hit Rate: {hit_rate}, Profit Factor: {profit_factor}, Sharpe Ratio: {sharpe_ratio}, Max Drawdown: {max_drawdown}, Total Profit: {total_profit}, Profit Percentage: {profit_percentage}%, Average Win to Average Loss ratio: {avg_win_to_avg_loss_ratio}, Sortino Ratio: {sortino_ratio}, Number of Trades: {num_trades}")
    metrics_dict_adx[n] = (hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades)

# Find the best n
best_n = max(metrics_dict_adx, key=lambda x: metrics_dict_adx[x][2])  # This line finds the n that gives the maximum Sharpe Ratio
print("The best window size is ", best_n)
# Run the backtest function with the best n to get the sma_signal column
df, hit_rate, profit_factor, sharpe_ratio, max_drawdown, total_profit, profit_percentage, avg_win_to_avg_loss_ratio, sortino_ratio, num_trades =backtest(df, best_n, starting_capital, stop_loss, upper_cap, lower_cap, transaction_cost, leverage, volume_percentage)
# Assuming metrics_df is your metrics report DataFrame
metrics_df = metrics_df.append({
    "Indicator":"ADX ",
    'Best_n': best_n,
    'Best_m': None,
    'Hit_Rate': hit_rate,
    'Profit_Factor': profit_factor,
    'Sharpe_Ratio': sharpe_ratio,
    'Max_Drawdown': max_drawdown,
    'Total_Profit': total_profit,
    'Profit_Percentage': profit_percentage,
    'Avg_Win_to_Avg_Loss_Ratio': avg_win_to_avg_loss_ratio,
    'Sortino_Ratio': sortino_ratio,
    'Num_Trades': num_trades
}, ignore_index=True)


For parameter: 2, Hit Rate: 0.10606060606060606, Profit Factor: 0.46376577383217726, Sharpe Ratio: -9.836756876564483, Max Drawdown: -256.65000000000055, Total Profit: -2873.1229677498986, Profit Percentage: -28.731229677498987%, Average Win to Average Loss ratio: -1.830340723402786, Sortino Ratio: -11.011498866608175, Number of Trades: 126
For parameter: 3, Hit Rate: 0.13636363636363635, Profit Factor: 0.5574209186123152, Sharpe Ratio: -8.243419630533543, Max Drawdown: -196.52760000000217, Total Profit: -2853.2274100998948, Profit Percentage: -28.53227410099895%, Average Win to Average Loss ratio: -2.0304718965491135, Sortino Ratio: -11.084506250758812, Number of Trades: 138
For parameter: 4, Hit Rate: 0.12121212121212122, Profit Factor: 0.33439332669012406, Sharpe Ratio: -14.010298112571391, Max Drawdown: -218.64999999999873, Total Profit: -2904.076143149914, Profit Percentage: -29.040761431499142%, Average Win to Average Loss ratio: -1.583867496564332, Sortino Ratio: -16.05621391626

In [72]:
metrics_df.head(10)

,Indicator,Best_n,Best_m,Hit_Rate,Profit_Factor,Sharpe_Ratio,Max_Drawdown,Total_Profit,Profit_Percentage,Avg_Win_to_Avg_Loss_Ratio,Sortino_Ratio,Num_Trades
0,SMA,527,None,0.181818,1.980214,7.296833,-411.40,1438.295600,14.382956,1.485161,10.209546,21
1,SMA_Crossover,564,4,0.166667,2.370887,8.357021,-384.80,1612.681500,16.126815,-1.724281,12.896507,19
2,EMA_Crossover,304,2,0.166667,2.370887,8.357021,-384.80,1612.681500,16.126815,-1.724281,12.896507,19
3,RSI,117,None,0.030303,-inf,8.440356,0.00,1939.418850,19.394189,NaN,NaN,2
4,ADX,46,None,0.287879,2.600435,11.038252,-361.35,2282.947000,22.829470,-1.913059,19.731586,37
5,Bollinger Bands,73,None,0.151515,2.617721,8.089186,-287.55,1686.323850,16.863239,-2.617721,14.582495,20
6,EMA,65,None,0.333333,2.599868,10.709183,-458.80,3360.744404,33.607444,2.430250,27.239468,45
7,EMA,38,None,0.303030,2.021919,8.524867,-457.60,2689.344690,26.893447,2.011663,18.962908,42
8,EMA,65,None,0.333333,2.599868,10.709183,-458.80,3360.744404,33.607444,2.430250,27.239468,45
9,EMA_Crossover,314,2,0.272727,1.909104,8.407585,-458.80,1434.463200,14.344632,-1.590920,12.410374,33


In [73]:
np.unique(df.ema_signal,return_counts=True)

(array([-1,  0,  1]), array([  45, 2407,   45]))

**Based on Local Extrema**

Hit Rate: 0.0, Profit Factor: nan, Sharpe Ratio: nan, Max Drawdown: 0, Total Profit: 0, Profit Percentage: 0.0%, Average Win to Average Loss ratio: nan, Sortino Ratio: nan, Number of Trades: 0


# **Report Obtained for each Indicator**
**The reports include these:**

(i) Net profit

(ii) Profit Factor

(iii) Sharpe Ratio

(iv) Hit Rate/Win rate

(v) Max Drawdown

(vi) Net Profit/ loss

(vii) Average Win to Average Loss ratio

(viii) Sortino Ratio

(ix)Number of Trades

In [ ]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'sma', 'sma_short',
       'sma_long', 'ema', 'ema_short', 'ema_long', 'rsi', 'std', 'upper_bb',
       'lower_bb', 'vwap', 'hl_diff', 'hc_diff', 'lc_diff', 'tr', '+dm', '-dm',
       'tr_ema', '+dm_ema', '-dm_ema', '+di', '-di', 'dx', 'adx', 'hl_avg',
       'range', 'upper_band', 'lower_band', 'upper_band_final',
       'lower_band_final', 'supertrend', 'macd_line', 'signal_line',
       'macd_histogram'],
      dtype='object')

# **Report Obtained using MaxVote Strategy for all Indicators**

# **Plots and Vizualizations for selective Indicators**

In [ ]:
df2=df.copy()
df2.reset_index(level=0, inplace=True)
df2.rename(columns={'index': 'Date'}, inplace=True)
# callable plotting function
def plot_df(columns, date_range):
    if columns:  # Check if any column is selected
        fig, ax = plt.subplots(figsize=(50, 20))  # Create a figure and a set of subplots
        df2.set_index('Date').loc[date_range[0]:date_range[1], list(columns)].plot(ax=ax)  # Plot on the specific axes
        ax.grid(True)
        ax.legend(fontsize="40")  # Increase the legend size

        font = {'weight' : 'bold',
                'size'   : 22}

        plt.rc('font', **font)
        plt.show()

# Getting column names from the dfFrame
column_names = [col for col in df2.columns.tolist() if col != 'Date']

# Here creating date range slider
dates = pd.date_range(df2['Date'].min(), df2['Date'].max(), freq='D')
options = [(date.strftime(' %d %b %Y '), date) for date in dates]
date_range_slider = SelectionRangeSlider(
    options=options,
    index=(0, len(options)-1),
    description='Dates',
    orientation='horizontal',
    layout={'width': '500px'}
)

# Plotting interactive plot
interact(plot_df, columns=SelectMultiple(options=column_names), date_range=date_range_slider)

interactive(children=(SelectMultiple(description='columns', options=('Open', 'High', 'Low', 'Close', 'Volume',…

<function __main__.plot_df(columns, date_range)>